In [ ]:
import os
import bilby
import numpy as np 
import itertools
import matplotlib
from glob import glob
import matplotlib.pyplot as plt

plt.style.use('default')
plt.rcParams.update({"text.usetex": True,
    "font.family": "sans-serif",
    "axes.formatter.use_mathtext": True,
    "axes.formatter.limits": (-3, 3)
})
%matplotlib inline

## Injections

In [ ]:
mchirpratio, snrratio = [.5, 1., 2.], [.5, 1.]
delta_tc = [-.1, -.05, -.03, -.02, -.01, .01, .02, .03, .05, .1]

combinations = list(itertools.product(mchirpratio, snrratio, delta_tc))

data_a, data_b = [], []
mchirp_a, mchirp_b = np.zeros(len(combinations)), np.zeros(len(combinations))
q_a, q_b = np.zeros(len(combinations)), np.zeros(len(combinations))
eta_a, eta_b = np.zeros(len(combinations)), np.zeros(len(combinations))
eff_spin_a, eff_spin_b = np.zeros(len(combinations)), np.zeros(len(combinations))
delta_tc, snr_a, snr_b = np.zeros(len(combinations)), np.zeros(len(combinations)), np.zeros(len(combinations))

for i in range(len(combinations)):
    data_a.append(np.load('../../output/pe_population/injections/SINGLES_A_{}_{}_{}.npy'.format(combinations[i][0], combinations[i][1], combinations[i][2]), allow_pickle=True).item())
    data_b.append(np.load('../../output/pe_population/injections/SINGLES_B_{}_{}_{}.npy'.format(combinations[i][0], combinations[i][1], combinations[i][2]), allow_pickle=True).item())

    mchirp_a[i], mchirp_b[i] = np.power(data_a[i]['mass_1']*data_a[i]['mass_2'], (3/5))/np.power(data_a[i]['mass_1']+data_a[i]['mass_2'], (1/5)), np.power(data_b[i]['mass_1']*data_b[i]['mass_2'], (3/5))/np.power(data_b[i]['mass_1']+data_b[i]['mass_2'], (1/5))
    q_a[i], q_b[i] = data_a[i]['mass_2']/data_a[i]['mass_1'], data_b[i]['mass_2']/data_b[i]['mass_1']
    eta_a[i], eta_b[i] = (data_a[i]['mass_1']*data_a[i]['mass_2'])/np.power(data_a[i]['mass_1']+data_a[i]['mass_2'], 2), (data_b[i]['mass_1']*data_b[i]['mass_2'])/np.power(data_b[i]['mass_1']+data_b[i]['mass_2'], 2)
    eff_spin_a[i], eff_spin_b[i] = data_a[i]['a_1'], data_b[i]['a_1']

    delta_tc[i] = data_b[i]['geocent_time'] - data_a[i]['geocent_time']
    snr_a[i], snr_b[i] = data_a[i]['snr_det'], data_b[i]['snr_det']

In [ ]:
fig, axes = plt.subplots(1, 1, figsize=(12, 9), subplot_kw={'projection': '3d'})
axes.scatter(snr_b/snr_a, mchirp_b/mchirp_a, delta_tc, s=100)
axes.set_xlabel('$\mathrm{SNR}_B/\mathrm{SNR}_A$')
axes.set_ylabel('$\mathcal{M}_B/\mathcal{M}_A$')
axes.set_zlabel('$\Delta t_c$')
axes.ticklabel_format(useOffset=False, style='plain')
axes.set_title('$\\mathcal{M}_B/\\mathcal{M}_A\\in[0.5,2], \ \\mathrm{SNR}_B/\\mathrm{SNR}_A\\in[0.5,2], \ \\Delta t_c\\in[-1,\ 1]s$')
plt.suptitle('Overlapping Population Injections')
plt.tight_layout()
plt.savefig('../../output/pe_population/pe/INJECTIONS.png')
plt.show()

## Unlensed Quasi-Circular Recovery

In [ ]:
mchirp = np.zeros(len(combinations))
for i in range(len(combinations)):
    fs = glob('../../output/pe_population/pe/PE_{}_{}_{}/result/*.json'.format(combinations[i][0], combinations[i][1], combinations[i][2]))
    if len(fs) == 0:
        print('File not found: PE_{}_{}_{}'.format(combinations[i][0], combinations[i][1], combinations[i][2]))
        mchirp[i] = np.nan
        continue
    result = bilby.result.read_in_result(filename=fs[0])
    
    mchirp[i] = result.posterior['chirp_mass'].median()

    plot1 = result.plot_corner(
        parameters=["chirp_mass", "mass_ratio"], save=False
    )
    plot1.axes[0].axvline(mchirp_a[i], color='r', linestyle='--')
    plot1.axes[0].axvline(mchirp_b[i], color='m', linestyle='--')
    plot1.axes[3].axvline(q_a[i], color='r', linestyle='--')
    plot1.axes[3].axvline(q_b[i], color='m', linestyle='--')
    plot1.axes[1].text(0.5, 0.5, '$\mathcal{{M}}_A={}$\n$\mathcal{{M}}_B={}$\n$q_A={}$\n$q_B={}$'.format(round(mchirp_a[i],2), round(mchirp_b[i],2), round(q_a[i],2), round(q_b[i], 2)), horizontalalignment='center', verticalalignment='center', transform=plot1.axes[1].transAxes, fontsize=15)
    plot1.suptitle('$\mathrm{{SNR}}_A/\mathrm{{SNR}}_B={}$\n$\mathcal{{M}}_A/\mathcal{{M}}_B={}\ \&\ \Delta t_c={}$'.format(round(snr_a[i]/snr_b[i],2), round(mchirp_a[i]/mchirp_b[i],2), round(delta_tc[i],3)), fontsize=20)
    plot1.set_size_inches(8, 8)
    plt.tight_layout()
    plt.savefig('../../output/pe_population/pe/corner/Mq_PE_{}_{}_{}.png'.format(combinations[i][0], combinations[i][1], combinations[i][2]))
    plt.close()
    
    plot2 = result.plot_corner(
        parameters=["mass_1", "mass_2"], save=False
    )
    plot2.axes[0].axvline(data_a[i]['mass_1'], color='r', linestyle='--')
    plot2.axes[0].axvline(data_b[i]['mass_1'], color='m', linestyle='--')
    plot2.axes[3].axvline(data_a[i]['mass_2'], color='r', linestyle='--')
    plot2.axes[3].axvline(data_b[i]['mass_2'], color='m', linestyle='--')
    plot2.axes[1].text(0.5, 0.5, '$m_{{1A}}={}$\n$m_{{1B}}={}$\n$m_{{2A}}={}$\n$m_{{2B}}={}$'.format(round(data_a[i]['mass_1'],2), round(data_b[i]['mass_1'],2), round(data_a[i]['mass_2'],2), round(data_b[i]['mass_2'], 2)), horizontalalignment='center', verticalalignment='center', transform=plot2.axes[1].transAxes, fontsize=15)
    plot2.suptitle('$\mathrm{{SNR}}_A/\mathrm{{SNR}}_B={}$\n$\mathcal{{M}}_A/\mathcal{{M}}_B={}\ \&\ \Delta t_c={}$'.format(round(snr_a[i]/snr_b[i],2), round(mchirp_a[i]/mchirp_b[i],2), round(delta_tc[i],3)), fontsize=20)
    plot2.set_size_inches(8, 8)
    plt.tight_layout()
    plt.savefig('../../output/pe_population/pe/corner/m1m2_PE_{}_{}_{}.png'.format(combinations[i][0], combinations[i][1], combinations[i][2]))
    plt.close()

In [ ]:
fs = glob('../../output/pe_population/pe/PE_{}_{}_{}/result/*.json'.format(combinations[i][0], combinations[i][1], combinations[i][2]))
result = bilby.result.read_in_result(filename=fs[0])
plot = result.plot_corner(
    parameters=["chirp_mass", "mass_ratio"], save=False
)
plot.axes[0].axvline(mchirp_a[i], color='r', linestyle='--')
plot.axes[0].axvline(mchirp_b[i], color='m', linestyle='--')
plot.axes[3].axvline(q_a[i], color='r', linestyle='--')
plot.axes[3].axvline(q_b[i], color='m', linestyle='--')
plot.axes[1].text(0.5, 0.5, '$\mathcal{{M}}_A={}$\n$\mathcal{{M}}_B={}$\n$q_A={}$\n$q_B={}$'.format(round(mchirp_a[i],2), round(mchirp_b[i],2), round(q_a[i],2), round(q_b[i], 2)), horizontalalignment='center', verticalalignment='center', transform=plot.axes[1].transAxes, fontsize=15)
plot.suptitle('$\mathrm{{SNR}}_A/\mathrm{{SNR}}_B={}$\n$\mathcal{{M}}_A/\mathcal{{M}}_B={}\ \&\ \Delta t_c={}$'.format(round(snr_a[i]/snr_b[i],2), round(mchirp_a[i]/mchirp_b[i],2), round(delta_tc[i],3)), fontsize=20)
plot.set_size_inches(8, 8)
plt.tight_layout()
plt.show()
plt.close()

In [ ]:
plot = result.plot_corner(
    parameters=["mass_1", "mass_2"], save=False
)
plot.axes[0].axvline(data_a[i]['mass_1'], color='r', linestyle='--')
plot.axes[0].axvline(data_b[i]['mass_1'], color='m', linestyle='--')
plot.axes[3].axvline(data_a[i]['mass_2'], color='r', linestyle='--')
plot.axes[3].axvline(data_b[i]['mass_2'], color='m', linestyle='--')
plot.axes[1].text(0.5, 0.5, '$m_{{1A}}={}$\n$m_{{1B}}={}$\n$m_{{2A}}={}$\n$m_{{2B}}={}$'.format(round(data_a[i]['mass_1'],2), round(data_b[i]['mass_1'],2), round(data_a[i]['mass_2'],2), round(data_b[i]['mass_2'], 2)), horizontalalignment='center', verticalalignment='center', transform=plot.axes[1].transAxes, fontsize=15)
plot.suptitle('$\mathrm{{SNR}}_A/\mathrm{{SNR}}_B={}$\n$\mathcal{{M}}_A/\mathcal{{M}}_B={}\ \&\ \Delta t_c={}$'.format(round(snr_a[i]/snr_b[i],2), round(mchirp_a[i]/mchirp_b[i],2), round(delta_tc[i],3)), fontsize=20)
plot.set_size_inches(8, 8)
plt.tight_layout()
plt.show()
plt.close()

### Cummulative Plots

In [ ]:
bias_mchirp_a = np.abs(mchirp-mchirp_a)/mchirp_a*100
bias_mchirp_b = np.abs(mchirp-mchirp_b)/mchirp_b*100

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 6), subplot_kw={'projection': '3d'})

scatter1 = axes[0].scatter(snr_b/snr_a, mchirp_b/mchirp_a, delta_tc, c=bias_mchirp_a, edgecolor=None, cmap='plasma_r', s=100)
clb1 = fig.colorbar(scatter1, ax=axes[0], shrink=0.7, pad=0.1)
clb1.ax.set_title('$\\%-$Bias$_A$')
clb1.ax.ticklabel_format(useOffset=False, style='plain')

axes[0].set_xlabel('$\mathrm{SNR}_B/\mathrm{SNR}_A$')
axes[0].set_ylabel('$\mathcal{M}_B/\mathcal{M}_A$')
axes[0].set_zlabel('$\Delta t_c$')
axes[0].set_title('Chirp Mass Bias$_A$ $\\beta_{\\mathcal{M}}=\\left(\\frac{\\mathcal{M}-\\mathcal{M}_A}{\\mathcal{M}_A}\\right)\\%$ values', loc='center')
axes[0].ticklabel_format(useOffset=False, style='plain')
axes[0].azim = 150

scatter2 = axes[1].scatter(snr_b/snr_a, mchirp_b/mchirp_a, delta_tc, c=bias_mchirp_b, edgecolor=None, cmap='plasma_r', s=100)
clb2 = fig.colorbar(scatter2, ax=axes[1], shrink=0.7, pad=0.1)
clb2.ax.set_title('$\\%-$Bias$_B$')
clb2.ax.ticklabel_format(useOffset=False, style='plain')

axes[1].set_xlabel('$\mathrm{SNR}_B/\mathrm{SNR}_A$')
axes[1].set_ylabel('$\mathcal{M}_B/\mathcal{M}_A$')
axes[1].set_zlabel('$\Delta t_c$')
axes[1].set_title('Chirp Mass Bias$_B$ $\\beta_{\\mathcal{M}}=\\left(\\frac{\\mathcal{M}-\\mathcal{M}_B}{\\mathcal{M}_B}\\right)\\%$ values', loc='center')
axes[1].ticklabel_format(useOffset=False, style='plain')
axes[1].azim = 150

fig.suptitle('PE: Injection parameters vs Recovered Chirp Mass Bias values', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.savefig('../../output/pe_population/output/MCHIRP_BIAS.png')
plt.show()